# Introduction

Emily brought a great point in my last lab meeting about how our dataset is paired. In our case DNA rep1 is paired to RNA rep1 and so forth. Most MPRA datasets are unlike this, so it would be advantagous to use this in our statistical analyses. 

Specifcally, I looked at DESeq2 and you can specify data that is paired. I want to test if using paired data improves region recall at all. 

# Call active regions using paired DESeq2 strategy. 

In [3]:
#read in libraries and counts matrix. Convert counts_df to a matrix. 
library(DESeq2)
library(tidyverse)
library(apeglm)

dir <- '/data/hodges_lab/ATAC-STARR_V2'

#Read in counts matrix
cts_df <- read_tsv(paste0(dir,'/data/activity/individual/cts_matricies/bin-method/GM12878inGM12878_counts_bin-method_50bp-bins_with-duplicates.tsv'), 
                   col_names = c("Bin_ID", "Chr", "Start", "End", "Strand", "Length", "DNA1", "DNA2", "RNA1", "RNA2"), 
                   col_types = "cciiciiiii", skip = 2)

#Convert cts to a matrix of integer counts with row names represented by Peak_ID.
cts_matrix <- as.matrix(dplyr::select(cts_df, Bin_ID, DNA1, DNA2, RNA1, RNA2) %>% column_to_rownames(var = "Bin_ID"))

Loading required package: GenomicRanges

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrde

In [4]:
#Prepare DESeq2 object for analysis. This is where the key difference in defining paired lies. 

#Prepare dataframe of sample info
condition <- c("DNA", "DNA", "RNA", "RNA")
paired_trial <- c("trial1", "trial2", "trial1", "trial2")
RNames <- c("DNA1", "DNA2", "RNA1", "RNA2")

coldata <- data.frame(row.names = RNames, condition, paired_trial)

head(coldata)

#Check that coldata matches cts. 
all(rownames(coldata) == colnames(cts_matrix))

#Set up experimental design for DEseq2
dds <- DESeqDataSetFromMatrix(countData = cts_matrix, colData = coldata, design = ~ paired_trial + condition)

,condition,paired_trial
,<chr>,<chr>
DNA1,DNA,trial1
DNA2,DNA,trial2
RNA1,RNA,trial1
RNA2,RNA,trial2


[1] TRUE

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


In [5]:
#Perform differential analysis and extract results. 

#info to make parallel
library(BiocParallel)
register(MulticoreParam(4))

#differential expression analysis. Use local. I tried all three, mean has a poor fit and parametric defaults to local anyway. 
dds <- DESeq(dds, fitType="local", parallel=TRUE, BPPARAM=MulticoreParam(4))

#Extract results: RNA/DNA. lfcShrink with apeglm. 
res <- lfcShrink(dds, coef="condition_RNA_vs_DNA", type="apeglm", parallel=TRUE, BPPARAM=MulticoreParam(4)) 

#Convert to df and add Bin_ID column from rownames. 
res_df <- as.data.frame(res) %>% rownames_to_column(var = "Bin_ID")

estimating size factors

estimating dispersions

gene-wise dispersion estimates: 4 workers

mean-dispersion relationship

final dispersion estimates, fitting model and testing: 4 workers

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



In [7]:
#Extract and save active & silent bins. 

#Obtain active bins using the following filters: padj < 0.1 and l2fc > 0. 
active <- filter(res_df, padj < 0.1) %>% filter(log2FoldChange > 0) 

#Add positional information back by joining with cts_df. Then format to bed (log2Fc as score) and sort by position.  
active_bed <- left_join(active, cts_df, by = c("Bin_ID" = "Bin_ID"), keep = FALSE) %>% 
           dplyr::select(Chr, Start, End, Bin_ID, log2FoldChange, Strand) %>% 
           dplyr::arrange(Chr, Start, End)

#Obtain silent bins using the following filters: padj < 0.1 and l2fc < 0. 
silent <- filter(res_df, padj < 0.1) %>% filter(log2FoldChange < 0) 

#Add positional information back by joining with cts_df. Then format to bed (log2Fc as score) and sort by position.  
silent_bed <- left_join(silent, cts_df, by = c("Bin_ID" = "Bin_ID"), keep = FALSE) %>% 
           dplyr::select(Chr, Start, End, Bin_ID, log2FoldChange, Strand) %>% 
           dplyr::arrange(Chr, Start, End)

#Save active and silent bed files. 
write_tsv(active_bed, paste0(dir, "/data/activity/individual/DESeq2_results/bin-method/GM12878inGM12878_active_0.1padj_50-bin_paired.bed"), col_names = FALSE)
write_tsv(silent_bed, paste0(dir, "/data/activity/individual/DESeq2_results/bin-method/GM12878inGM12878_silent_0.1padj_50-bin_paired.bed"), col_names = FALSE)

In [1]:
DATA_DIR='/data/hodges_lab/ATAC-STARR_V2/data/activity/individual/DESeq2_results/bin-method'

module restore tools

bedtools merge -c 5 -o mean -i ${DATA_DIR}/GM12878inGM12878_active_0.1padj_50-bin_paired.bed | wc -l 
bedtools merge -c 5 -o mean -i ${DATA_DIR}/GM12878inGM12878_silent_0.1padj_50-bin_paired.bed | wc -l 

Restoring modules from user's tools
7050
16249


# Summary

This actually resulted in fewer regions called as active. This seems like a dead end. 